In [ ]:
import os
import json
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# Define your image and annotation directories
image_dir = "C:\\Users\\asus\\Downloads\\corrosion-images-paper\\corrosion-images\\images\\train\\images"
annotation_dir = "C:\\Users\\asus\\Downloads\\corrosion-images-paper\\corrosion-images\\Annotation-json-format\\train\\label"

# Create directories to store the RGB masks
output_dir = "C:\\Users\\asus\\Downloads\\folder"
os.makedirs(output_dir, exist_ok=True)



color_map ={"BackGround": (0, 0, 0), "Fair": (138, 192, 208), "Poor": (21, 179, 42), "Severe": (189, 148, 86)}
#blue green yellow
# Function to create RGB mask from annotation
def create_rgb_mask(annotation, color_map, image_size):
    width, height = image_size
    rgb_mask = np.zeros((height, width, 3), dtype=np.uint8)
    rgb_mask_pil = Image.fromarray(rgb_mask)
    draw = ImageDraw.Draw(rgb_mask_pil)
    for shape in annotation['shapes']:
        label = shape['label']
        points = shape['points']
        polygon = [(x, y) for x, y in points]
        color = color_map.get(label, (0, 0, 0))  # Default color is black if not found in color_map
        draw.polygon(polygon, fill=color)
    rgb_mask = np.array(rgb_mask_pil)

    return rgb_mask


# Iterate through each annotation file

i=0

for annotation_file in os.listdir(annotation_dir):
    if annotation_file.endswith('.json'):
        # Load annotation
        annotation_path = os.path.join(annotation_dir, annotation_file)
        with open(annotation_path, 'r') as f:
            annotation_data = json.load(f)

        # Load corresponding image
        image_name = annotation_file[:-5] +".png"
        print(annotation_file,image_name)

        image_path = os.path.join(image_dir, image_name)

        print("hii",image_path,"h",annotation_path,"si")
       
        image = Image.open(image_path)
        image_size = image.size

        # Create RGB mask from annotation
        rgb_mask = create_rgb_mask(annotation_data, color_map, image_size)

        # Save RGB mask
        output_path = os.path.join(output_dir, f"{os.path.splitext(annotation_file)[0]}_mask.png")
        print(output_path)
        Image.fromarray(rgb_mask).save(output_path)

        # Display image and RGB mask
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(rgb_mask)
        plt.title('RGB Mask')
        plt.axis('off')

        plt.show()
